In [2]:
# 모듈 import
import sys
import os

# 현재 노트북 파일의 절대 경로
current_path = os.path.abspath('')

# 상위 폴더 경로
parent_path = os.path.dirname(current_path)

# 상위 폴더를 sys.path에 추가
sys.path.append(parent_path)

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [13]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#print(cfg.data.train)
#print(cfg.model)
#cfg = Config.fromfile('/data/ephemeral/home/Jihwan/level2-objectdetection-cv-07/mmdetection/ssd_experiment/ssd300_coco.py')
cfg = Config.fromfile('/data/ephemeral/home/Jihwan/level2-objectdetection-cv-07/mmdetection/configs/ssd/ssd300_coco.py')



root='/data/ephemeral/home/dataset/'

# dataset config 수정
cfg.data.train.dataset.classes = classes
cfg.data.train.dataset.img_prefix = root
cfg.data.train.dataset.ann_file = root + 'train.json' # train json 정보
cfg.data.train.dataset.pipeline[4]['img_scale'] = (512,512) # Resize ssd
cfg.data.train.times = 1
cfg.data.img_norm_cfg = dict(mean=[123.675, 116.28, 110.53], std=[60, 59, 61], to_rgb=True)
#cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize faster-rcnn

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/ssd300_coco_trash_1'

cfg.model.bbox_head.num_classes = 10
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [14]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [15]:
# dataset 확인
datasets[0]

In [17]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-11 18:52:16,943 - mmcv - INFO - initialize SSDVGG with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://vgg16_caffe'}
2024-10-11 18:52:16,944 - mmcv - INFO - load model from: open-mmlab://vgg16_caffe
2024-10-11 18:52:16,945 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://vgg16_caffe
Downloading: "https://download.openmmlab.com/pretrain/third_party/vgg16_caffe-292e1171.pth" to /data/ephemeral/home/.cache/torch/hub/checkpoints/vgg16_caffe-292e1171.pth


  0%|          | 0.00/78.1M [00:00<?, ?B/s]

2024-10-11 18:52:20,339 - mmcv - INFO - initialize SSDNeck with init_cfg [{'type': 'Xavier', 'distribution': 'uniform', 'layer': 'Conv2d'}, {'type': 'Constant', 'val': 1, 'layer': 'BatchNorm2d'}, {'type': 'Constant', 'val': 20, 'override': {'name': 'l2_norm'}}]
2024-10-11 18:52:20,361 - mmcv - INFO - initialize SSDHead with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform', 'bias': 0}
2024-10-11 18:52:20,378 - mmcv - INFO - 
backbone.features.0.weight - torch.Size([64, 3, 3, 3]): 
PretrainedInit: load from open-mmlab://vgg16_caffe 
 
2024-10-11 18:52:20,379 - mmcv - INFO - 
backbone.features.0.bias - torch.Size([64]): 
PretrainedInit: load from open-mmlab://vgg16_caffe 
 
2024-10-11 18:52:20,380 - mmcv - INFO - 
backbone.features.2.weight - torch.Size([64, 64, 3, 3]): 
PretrainedInit: load from open-mmlab://vgg16_caffe 
 
2024-10-11 18:52:20,380 - mmcv - INFO - 
backbone.features.2.bias - torch.Size([64]): 
PretrainedInit: load from open-mmlab://vgg16_caffe 
 
20

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-10-11 18:52:42,417 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-11 18:52:42,419 - mmdet - INFO - Start running, host: root@instance-12171, work_dir: /data/ephemeral/home/Jihwan/level2-objectdetection-cv-07/mmdetection/ssd_experiment/work_dirs/ssd300_coco_trash_1
2024-10-11 18:52:42,420 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_N